You are an expert Python developer specializing in the Databricks environment. Your task is to create a complete Python script to be executed within a Databricks notebook. The script must perform the following operations:
1.	Data Retrieval from SpaceX API:
o	Interact with the SpaceX v3 REST API (https://api.spacexdata.com/v3).
o	Retrieve data from one specific endpoint: 
	All launches: https://api.spacexdata.com/v3/launches
o	Handle potential errors during the API calls (e.g., timeouts, non-200 status codes).
2.	Aggregate Operation:
o	Perform a simple "aggregate" operation on the retrieved launch data.
o	Aggregation Logic: Calculate the total number of launches for each year.
o	Return a print with the number of aggregate launches for each year
3.	Control Parameters and Debugging:
o	Include a variable at the beginning of the script to define the API endpoint URL, making it easily modifiable: 
	API_ENDPOINT_URL = "https://api.spacexdata.com/v3/launches"
o	Use Python's standard logging module to provide informative output during execution. Configure logging to display messages at the INFO level.
o	Log key messages such as: starting data retrieval, number of launches retrieved, starting aggregation, aggregation complete, number of years aggregated, starting upload to httpbin, upload outcome.
4.	Execution Time Measurement:
o	Code Execution Time: Measure the time taken to perform the main operations (data retrieval + aggregation). Print this time after the aggregation operation is complete.
o	Pipeline Execution Time: Measure the total execution time of the entire script (from the beginning until after the upload to httpbin). Print this total time at the end of the script. Use Python's time module.
5.	Upload Result:
o	Take the resulting aggregated summary dictionary (year counts) from the aggregate operation.
o	Serialize it into JSON format.
o	Make an HTTP POST request to the https://httpbin.org/post endpoint, sending the resulting aggregated JSON data in the request body.
o	Verify the response from httpbin.org (e.g., check the status code) and log the outcome of the upload operation.


In [0]:
# Databricks Notebook Script

# COMMAND ----------

# Imports
import requests
import logging
import time
import json
from collections import Counter

# COMMAND ----------

# Control Parameters
API_ENDPOINT_URL = "https://api.spacexdata.com/v3/launches"
UPLOAD_URL = "https://httpbin.org/post"

# Setup Logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")
logger = logging.getLogger()

# COMMAND ----------

# Start measuring total pipeline execution time
pipeline_start_time = time.time()

# COMMAND ----------

def retrieve_spacex_launches(api_url):
    """Retrieve SpaceX launches from API"""
    logger.info("Starting data retrieval from SpaceX API...")
    try:
        response = requests.get(api_url, timeout=10)
        response.raise_for_status()
        launches = response.json()
        logger.info(f"Data retrieval successful: Retrieved {len(launches)} launches.")
        return launches
    except requests.exceptions.RequestException as e:
        logger.error(f"Error retrieving data from SpaceX API: {e}")
        raise

# COMMAND ----------

def aggregate_launches_by_year(launches):
    """Aggregate number of launches per year"""
    logger.info("Starting aggregation of launches by year...")
    years = [launch.get('launch_year') for launch in launches if launch.get('launch_year')]
    year_counts = dict(Counter(years))
    logger.info(f"Aggregation complete: Aggregated launches for {len(year_counts)} years.")
    return year_counts

# COMMAND ----------

def upload_aggregation_result(upload_url, data):
    """Upload aggregated result to httpbin.org"""
    logger.info("Starting upload of aggregated result to httpbin.org...")
    try:
        headers = {"Content-Type": "application/json"}
        response = requests.post(upload_url, headers=headers, data=json.dumps(data), timeout=10)
        response.raise_for_status()
        logger.info(f"Upload successful: Response status code {response.status_code}.")
    except requests.exceptions.RequestException as e:
        logger.error(f"Error uploading data to httpbin.org: {e}")
        raise

# COMMAND ----------

# Main Execution

# Measure time for retrieval + aggregation
main_start_time = time.time()

# Step 1: Retrieve Data
launch_data = retrieve_spacex_launches(API_ENDPOINT_URL)

# Step 2: Aggregate Data
launches_per_year = aggregate_launches_by_year(launch_data)

# Print aggregation result
print("Number of SpaceX Launches Per Year:")
for year, count in sorted(launches_per_year.items()):
    print(f"{year}: {count}")

# Measure and print main operation time
main_end_time = time.time()
main_operation_duration = main_end_time - main_start_time
print(f"\nMain operations (data retrieval + aggregation) completed in {main_operation_duration:.2f} seconds.")

# COMMAND ----------

# Step 3: Upload result to httpbin
upload_aggregation_result(UPLOAD_URL, launches_per_year)

# COMMAND ----------

# End of pipeline - measure total time
pipeline_end_time = time.time()
total_pipeline_duration = pipeline_end_time - pipeline_start_time

print(f"\nTotal pipeline execution completed in {total_pipeline_duration:.2f} seconds.")
